In [1]:
import pandas as pd
from utils import *

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


ModuleNotFoundError: No module named 'utils'

<h1>Cleaning a CSV</h1>

Read in a CSV

In [ ]:
# load a dataframe
infile_path = "../data/raw/silver_shelby_county_properties_municipality_breakdown - s2_in_lieu_contracts_by_owner.csv"
df = pd.read_csv(infile_path)
df

In [ ]:
# Clean the column headers
df = clean_headers(df, [remove_special_for_words, truncate, snake_case, lower_case])
df.columns

In [ ]:
# Clean the string columns
string_columns = ["owner", 
                  "location"]
df = clean_columns(df, string_columns, [remove_special_for_words, truncate, lower_case])
df

In [ ]:
df.columns

In [ ]:
# Clean the numeric columns
numeric_columns = ['assmt_taxes', 'pilot_contract_amt',
       'pilot_debt_service_amt', 'difference']
df = clean_columns(df, numeric_columns, [remove_special_for_numbers])
df

In [ ]:
# Clean the date columns

# Because this data has an idiosyncratic "end date" value of "End of Loan", 
# we must define a dataset-specific function
def cast_end_date_to_datetime(entry: str | Number):
    try:
        return pd.to_datetime(str(entry))
    except ValueError:
        return entry
df["begin_date"] = pd.to_datetime(df["begin_date"])
df["end_date"] = df["end_date"].apply(lambda x: cast_end_date_to_datetime(x))
df

In [ ]:
# Clean the list columns

# Pandas always reads in lists as strings, so they will ultimately be converted to a string type, but putting them in lists firsts will allow us to convert them back to lists more quickly the next time we read them in because the list brackets will be included 
# in the output string

list_columns = ["pilot_parcel_no", "underlying_parcel_number"]
for col in list_columns:
    df[col] = df[col].apply(lambda x: str(x).split(","))
df

In [ ]:
# Create a unique index column based on the values in the identifying column
# Note: Ids are based on a deterministic hash, which means they are 
# reproducible even if the function is run by a different user at a different time,
# as long as the string in the identifying column is the same. 

identifying_columns = ["owner"]
df = set_unique_index(df=df, columns_to_hash = identifying_columns, index_length_limit=12)
df

In [ ]:
# Check the data types of each variable
df.dtypes

In [ ]:
# Fix the data type of each variable

names_to_types = {'owner': 'string',
                  'location': 'string',
                  'pilot_parcel_no': 'string', 
                  'underlying_parcel_number': 'string',
                  'begin_date': 'datetime64[ns]', 
                  'end_date': 'object',
                  'assmt_taxes': 'float64',
                  'pilot_contract_amt': 'float64',
       'pilot_debt_service_amt': 'float64', 
                  'difference': 'float64'}
df = cast_data_types(df, names_to_types)
df.dtypes

In [ ]:
# Save the now-clean csv and convert it to utf-8 format
outfile_path = "../data/processed/s2_in_lieu_contracts_by_owner.csv"
df.to_csv(outfile_path)
convert_utf8(original_file_path=outfile_path, new_file_path=outfile_path)